<a href="https://www.kaggle.com/code/prasadposture121/randomforestclassifier?scriptVersionId=108573979" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

# RandomForestClassifier() for Everything

I'm trying to create an application which takes train data (to train the model) and test data (to make predictions on) as the inputs, and gives output in the form of accuracy of the and predictions it made on test data.

My aim is to make the code reusable so that it works for any data given to it, and the little we have to do is to give those required inputs manually and the applications does all the needful which includes filling missing values, encoding categorical attributes, dropping unneccessary columns and much more.

I'm open for all types of suggestions and reviews, feel free to comment.   
Thank You!

# House Price Predictions
We will the re-usable code for making predictions on housing price. We will also see more examples.

In [12]:
#importing requied libraries
import pandas as pd
import numpy as np
import warnings
print('Libraries Imported') #making sure the libraries are imported
#filtering the warnings cuz presence of warnigs would make the output messier
warnings.filterwarnings('ignore')


#loading the train data (with user input)
train_data=input('Enter the train data :')
#for application this has to be like 'upload your file' kinda option
train=pd.read_csv(train_data)

#loading the test data (with user input)
test_data=input('Enter the test data :')
test=pd.read_csv(test_data)

#loading the sample submission to save our predictions
#the sample submissions is given all the time along with test and train data
#but in case it isnt given we need to put it in try-except block
#we will have to make changes in the saving method as well
sample_data=input('Enter the sample submission :')
sample=pd.read_csv(sample_data)
print('Data Loaded!') #making sure the data is loaded

#getting stastical information of the numeric variables
#storing it into a dataframe for using it later for filling missing values and 
#seperating numeric columns form categorical columns
train_describe=pd.DataFrame(train.describe())

#doing the same for test data
test_describe=pd.DataFrame(test.describe())

#describing with category
#we will put it in try...except block since some of the data many not have categorical attributes
#saving the stastical information will be useful while filling the missing values
try:
    train_describe_cat=pd.DataFrame(train.describe(include=['O']))
except Exception as e:
    pass
try:
    test_describe_cat=pd.DataFrame(test.describe(include=['O']))
except Exception as e:
    pass

#We will fill these missing values -
#using median for numerical data
#using top for categorical data

#getting the list of numeric columns from stastical description
train_describe_columns=list(train_describe.columns)
test_describe_columns=list(test_describe.columns)

#getting the list of categorical columns from the stastical description
try:
    train_describe_cat_columns=list(train_describe_cat.columns)
except:
    pass
try:
    test_describe_cat_columns=list(test_describe_cat.columns)
except:
    pass

#filling the missing values for numerical values with the median
for i in train_describe_columns:
    train[i].fillna(train[i].median(), inplace=True)
for i in test_describe_columns:
    test[i].fillna(test[i].median(), inplace=True)
    
#filling the missing values for categorical values with the mostly reccurring value
try:
    for i in train_describe_cat_columns:
        train[i].fillna(train_describe_cat.iloc[2,train_describe_cat_columns.index(i)], inplace=True)
    for i in test_describe_cat_columns:
        test[i].fillna(test_describe_cat.iloc[2,test_describe_cat_columns.index(i)], inplace=True)
except:
    pass
print('We filled the missing values!') #making sure the missing values are filled

#setting ID's
#We will put it in try except block since sometimes we dont have seperate data
try:
    for i in list(train.columns):
        if 'Id' in i: #using the fact that infact Id have string 'Id' in their name eg PassengerID.
            Id=i
            print('The Id of data is :', Id)
        else:
            pass
    train.set_index(Id ,inplace=True)
    test.set_index(Id ,inplace=True)
except:
    pass

#Label Encoding
#Labelling the categorical values with numbers so that machine could understand it
#putting it try-except block because we may not have categorical values
try:
    if Id in train_describe_cat_columns:
        train_describe_cat_columns.remove(Id)
    from sklearn.preprocessing import LabelEncoder
    for i in train_describe_cat_columns:
        le=LabelEncoder()
        arr=np.concatenate((train[i], test[i])).astype(str)
        le.fit(arr)
        train[i]=le.transform(train[i].astype(str))
        test[i]=le.transform(test[i].astype(str))
except:
    pass

#Getting the target variable aka the variable we gonna predict
#here we are using the fact that the target variable wouldn't be present in the test data
a=set(train.columns)
b=set(test.columns)
c=list(a-b)
label=c[0]
print('Target variable is :', label) #showing the target variable

#assiging dependent and independent variables
X=train.drop(label,axis=1)
y=train[label]

#train_test_split()
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.25, random_state=4)

#modelling
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier()
rf.fit(X_train,y_train)
pred_y=rf.predict(X_val)
from sklearn.metrics import accuracy_score
print('The accuracy is :',accuracy_score(y_val.values,pred_y))

#making predictions on test data
y_pred=rf.predict(test)

#saving the submissions in the form of pandas dataframe
#submission=pd.DataFrame({label:y_pred},index=test.index) : freq used in older versions
sample[label]=y_pred
file_name=input('Enter the title for saving predictions :')
submission.to_csv(file_name)
print('SUBMISSIONS SAVED SUCCESSFULLY!!!')
sample.head()#making sure that the submission is saved.

Libraries Imported


Enter the train data : ../input/house-prices-advanced-regression-techniques/train.csv
Enter the test data : ../input/house-prices-advanced-regression-techniques/test.csv
Enter the sample submission : ../input/house-prices-advanced-regression-techniques/sample_submission.csv


Data Loaded!
We filled the missing values!
The Id of data is : Id
Target variable is : SalePrice
The accuracy is : 0.010958904109589041


Enter the title for saving predictions : house_pricing.csv


SUBMISSIONS SAVED SUCCESSFULLY!!!


,Id,SalePrice
0,1461,110000
1,1462,158000
2,1463,192000
3,1464,192000
4,1465,153900


In [11]:
sample.head()

,Id,SalePrice
0,1461,110000
1,1462,151500
2,1463,192000
3,1464,178000
4,1465,180000


# Titanic
Now will use the same code for making predictions on titanic dataset.

In [14]:
#importing requied libraries
import pandas as pd
import numpy as np
import warnings
print('Libraries Imported') #making sure the libraries are imported
#filtering the warnings cuz presence of warnigs would make the output messier
warnings.filterwarnings('ignore')


#loading the train data (with user input)
train_data=input('Enter the train data :')
#for application this has to be like 'upload your file' kinda option
train=pd.read_csv(train_data)

#loading the test data (with user input)
test_data=input('Enter the test data :')
test=pd.read_csv(test_data)

#loading the sample submission to save our predictions
#the sample submissions is given all the time along with test and train data
#but in case it isnt given we need to put it in try-except block
#we will have to make changes in the saving method as well
sample_data=input('Enter the sample submission :')
sample=pd.read_csv(sample_data)
print('Data Loaded!') #making sure the data is loaded

#getting stastical information of the numeric variables
#storing it into a dataframe for using it later for filling missing values and 
#seperating numeric columns form categorical columns
train_describe=pd.DataFrame(train.describe())

#doing the same for test data
test_describe=pd.DataFrame(test.describe())

#describing with category
#we will put it in try...except block since some of the data many not have categorical attributes
#saving the stastical information will be useful while filling the missing values
try:
    train_describe_cat=pd.DataFrame(train.describe(include=['O']))
except Exception as e:
    pass
try:
    test_describe_cat=pd.DataFrame(test.describe(include=['O']))
except Exception as e:
    pass

#We will fill these missing values -
#using median for numerical data
#using top for categorical data

#getting the list of numeric columns from stastical description
train_describe_columns=list(train_describe.columns)
test_describe_columns=list(test_describe.columns)

#getting the list of categorical columns from the stastical description
try:
    train_describe_cat_columns=list(train_describe_cat.columns)
except:
    pass
try:
    test_describe_cat_columns=list(test_describe_cat.columns)
except:
    pass

#filling the missing values for numerical values with the median
for i in train_describe_columns:
    train[i].fillna(train[i].median(), inplace=True)
for i in test_describe_columns:
    test[i].fillna(test[i].median(), inplace=True)
    
#filling the missing values for categorical values with the mostly reccurring value
try:
    for i in train_describe_cat_columns:
        train[i].fillna(train_describe_cat.iloc[2,train_describe_cat_columns.index(i)], inplace=True)
    for i in test_describe_cat_columns:
        test[i].fillna(test_describe_cat.iloc[2,test_describe_cat_columns.index(i)], inplace=True)
except:
    pass
print('We filled the missing values!') #making sure the missing values are filled

#setting ID's
#We will put it in try except block since sometimes we dont have seperate data
try:
    for i in list(train.columns):
        if 'Id' in i: #using the fact that infact Id have string 'Id' in their name eg PassengerID.
            Id=i
            print('The Id of data is :', Id)
        else:
            pass
    train.set_index(Id ,inplace=True)
    test.set_index(Id ,inplace=True)
except:
    pass

#Label Encoding
#Labelling the categorical values with numbers so that machine could understand it
#putting it try-except block because we may not have categorical values
try:
    if Id in train_describe_cat_columns:
        train_describe_cat_columns.remove(Id)
    from sklearn.preprocessing import LabelEncoder
    for i in train_describe_cat_columns:
        le=LabelEncoder()
        arr=np.concatenate((train[i], test[i])).astype(str)
        le.fit(arr)
        train[i]=le.transform(train[i].astype(str))
        test[i]=le.transform(test[i].astype(str))
except:
    pass

#Getting the target variable aka the variable we gonna predict
#here we are using the fact that the target variable wouldn't be present in the test data
a=set(train.columns)
b=set(test.columns)
c=list(a-b)
label=c[0]
print('Target variable is :', label) #showing the target variable

#assiging dependent and independent variables
X=train.drop(label,axis=1)
y=train[label]

#train_test_split()
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.25, random_state=4)

#modelling
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier()
rf.fit(X_train,y_train)
pred_y=rf.predict(X_val)
from sklearn.metrics import accuracy_score
print('The accuracy is :',accuracy_score(y_val.values,pred_y))

#making predictions on test data
y_pred=rf.predict(test)

#saving the submissions in the form of pandas dataframe
#submission=pd.DataFrame({label:y_pred},index=test.index) : freq used in older versions
sample[label]=y_pred
file_name=input('Enter the title for saving predictions :')
submission.to_csv(file_name)
print('SUBMISSIONS SAVED SUCCESSFULLY!!!')
sample.head()#making sure that the submission is saved.

Libraries Imported


Enter the train data : ../input/titanic/train.csv
Enter the test data : ../input/titanic/test.csv
Enter the sample submission : ../input/titanic/gender_submission.csv


Data Loaded!
We filled the missing values!
The Id of data is : PassengerId
Target variable is : Survived
The accuracy is : 0.874439461883408


Enter the title for saving predictions : titanic.csv


SUBMISSIONS SAVED SUCCESSFULLY!!!


,PassengerId,Survived
0,892,0
1,893,0
2,894,0
3,895,0
4,896,1


# Digit Recognizer
Let's try our code on something which doesn't have categorical data or missing values

In [15]:
#importing requied libraries
import pandas as pd
import numpy as np
import warnings
print('Libraries Imported') #making sure the libraries are imported
#filtering the warnings cuz presence of warnigs would make the output messier
warnings.filterwarnings('ignore')


#loading the train data (with user input)
train_data=input('Enter the train data :')
#for application this has to be like 'upload your file' kinda option
train=pd.read_csv(train_data)

#loading the test data (with user input)
test_data=input('Enter the test data :')
test=pd.read_csv(test_data)

#loading the sample submission to save our predictions
#the sample submissions is given all the time along with test and train data
#but in case it isnt given we need to put it in try-except block
#we will have to make changes in the saving method as well
sample_data=input('Enter the sample submission :')
sample=pd.read_csv(sample_data)
print('Data Loaded!') #making sure the data is loaded

#getting stastical information of the numeric variables
#storing it into a dataframe for using it later for filling missing values and 
#seperating numeric columns form categorical columns
train_describe=pd.DataFrame(train.describe())

#doing the same for test data
test_describe=pd.DataFrame(test.describe())

#describing with category
#we will put it in try...except block since some of the data many not have categorical attributes
#saving the stastical information will be useful while filling the missing values
try:
    train_describe_cat=pd.DataFrame(train.describe(include=['O']))
except Exception as e:
    pass
try:
    test_describe_cat=pd.DataFrame(test.describe(include=['O']))
except Exception as e:
    pass

#We will fill these missing values -
#using median for numerical data
#using top for categorical data

#getting the list of numeric columns from stastical description
train_describe_columns=list(train_describe.columns)
test_describe_columns=list(test_describe.columns)

#getting the list of categorical columns from the stastical description
try:
    train_describe_cat_columns=list(train_describe_cat.columns)
except:
    pass
try:
    test_describe_cat_columns=list(test_describe_cat.columns)
except:
    pass

#filling the missing values for numerical values with the median
for i in train_describe_columns:
    train[i].fillna(train[i].median(), inplace=True)
for i in test_describe_columns:
    test[i].fillna(test[i].median(), inplace=True)
    
#filling the missing values for categorical values with the mostly reccurring value
try:
    for i in train_describe_cat_columns:
        train[i].fillna(train_describe_cat.iloc[2,train_describe_cat_columns.index(i)], inplace=True)
    for i in test_describe_cat_columns:
        test[i].fillna(test_describe_cat.iloc[2,test_describe_cat_columns.index(i)], inplace=True)
except:
    pass
print('We filled the missing values!') #making sure the missing values are filled

#setting ID's
#We will put it in try except block since sometimes we dont have seperate data
try:
    for i in list(train.columns):
        if 'Id' in i: #using the fact that infact Id have string 'Id' in their name eg PassengerID.
            Id=i
            print('The Id of data is :', Id)
        else:
            pass
    train.set_index(Id ,inplace=True)
    test.set_index(Id ,inplace=True)
except:
    pass

#Label Encoding
#Labelling the categorical values with numbers so that machine could understand it
#putting it try-except block because we may not have categorical values
try:
    if Id in train_describe_cat_columns:
        train_describe_cat_columns.remove(Id)
    from sklearn.preprocessing import LabelEncoder
    for i in train_describe_cat_columns:
        le=LabelEncoder()
        arr=np.concatenate((train[i], test[i])).astype(str)
        le.fit(arr)
        train[i]=le.transform(train[i].astype(str))
        test[i]=le.transform(test[i].astype(str))
except:
    pass

#Getting the target variable aka the variable we gonna predict
#here we are using the fact that the target variable wouldn't be present in the test data
a=set(train.columns)
b=set(test.columns)
c=list(a-b)
label=c[0]
print('Target variable is :', label) #showing the target variable

#assiging dependent and independent variables
X=train.drop(label,axis=1)
y=train[label]

#train_test_split()
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.25, random_state=4)

#modelling
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier()
rf.fit(X_train,y_train)
pred_y=rf.predict(X_val)
from sklearn.metrics import accuracy_score
print('The accuracy is :',accuracy_score(y_val.values,pred_y))

#making predictions on test data
y_pred=rf.predict(test)

#saving the submissions in the form of pandas dataframe
#submission=pd.DataFrame({label:y_pred},index=test.index) : freq used in older versions
sample[label]=y_pred
file_name=input('Enter the title for saving predictions :')
submission.to_csv(file_name)
print('SUBMISSIONS SAVED SUCCESSFULLY!!!')
sample.head()#making sure that the submission is saved.

Libraries Imported


Enter the train data : ../input/digit-recognizer/train.csv
Enter the test data : ../input/digit-recognizer/test.csv
Enter the sample submission : ../input/digit-recognizer/sample_submission.csv


Data Loaded!
We filled the missing values!
Target variable is : label
The accuracy is : 0.962


Enter the title for saving predictions : digit_classifier.csv


SUBMISSIONS SAVED SUCCESSFULLY!!!


,ImageId,Label,label
0,1,0,2
1,2,0,0
2,3,0,9
3,4,0,9
4,5,0,3


suggested changes:
1. delete pre-existing label or you could just take the second column index of sample submission

# Spaceship Titanic
This is similiar as that of titanic, but this had me made special changes in the code to make it work for any general dataset.

In [16]:
#importing requied libraries
import pandas as pd
import numpy as np
import warnings
print('Libraries Imported') #making sure the libraries are imported
#filtering the warnings cuz presence of warnigs would make the output messier
warnings.filterwarnings('ignore')


#loading the train data (with user input)
train_data=input('Enter the train data :')
#for application this has to be like 'upload your file' kinda option
train=pd.read_csv(train_data)

#loading the test data (with user input)
test_data=input('Enter the test data :')
test=pd.read_csv(test_data)

#loading the sample submission to save our predictions
#the sample submissions is given all the time along with test and train data
#but in case it isnt given we need to put it in try-except block
#we will have to make changes in the saving method as well
sample_data=input('Enter the sample submission :')
sample=pd.read_csv(sample_data)
print('Data Loaded!') #making sure the data is loaded

#getting stastical information of the numeric variables
#storing it into a dataframe for using it later for filling missing values and 
#seperating numeric columns form categorical columns
train_describe=pd.DataFrame(train.describe())

#doing the same for test data
test_describe=pd.DataFrame(test.describe())

#describing with category
#we will put it in try...except block since some of the data many not have categorical attributes
#saving the stastical information will be useful while filling the missing values
try:
    train_describe_cat=pd.DataFrame(train.describe(include=['O']))
except Exception as e:
    pass
try:
    test_describe_cat=pd.DataFrame(test.describe(include=['O']))
except Exception as e:
    pass

#We will fill these missing values -
#using median for numerical data
#using top for categorical data

#getting the list of numeric columns from stastical description
train_describe_columns=list(train_describe.columns)
test_describe_columns=list(test_describe.columns)

#getting the list of categorical columns from the stastical description
try:
    train_describe_cat_columns=list(train_describe_cat.columns)
except:
    pass
try:
    test_describe_cat_columns=list(test_describe_cat.columns)
except:
    pass

#filling the missing values for numerical values with the median
for i in train_describe_columns:
    train[i].fillna(train[i].median(), inplace=True)
for i in test_describe_columns:
    test[i].fillna(test[i].median(), inplace=True)
    
#filling the missing values for categorical values with the mostly reccurring value
try:
    for i in train_describe_cat_columns:
        train[i].fillna(train_describe_cat.iloc[2,train_describe_cat_columns.index(i)], inplace=True)
    for i in test_describe_cat_columns:
        test[i].fillna(test_describe_cat.iloc[2,test_describe_cat_columns.index(i)], inplace=True)
except:
    pass
print('We filled the missing values!') #making sure the missing values are filled

#setting ID's
#We will put it in try except block since sometimes we dont have seperate data
try:
    for i in list(train.columns):
        if 'Id' in i: #using the fact that infact Id have string 'Id' in their name eg PassengerID.
            Id=i
            print('The Id of data is :', Id)
        else:
            pass
    train.set_index(Id ,inplace=True)
    test.set_index(Id ,inplace=True)
except:
    pass

#Label Encoding
#Labelling the categorical values with numbers so that machine could understand it
#putting it try-except block because we may not have categorical values
try:
    if Id in train_describe_cat_columns:
        train_describe_cat_columns.remove(Id)
    from sklearn.preprocessing import LabelEncoder
    for i in train_describe_cat_columns:
        le=LabelEncoder()
        arr=np.concatenate((train[i], test[i])).astype(str)
        le.fit(arr)
        train[i]=le.transform(train[i].astype(str))
        test[i]=le.transform(test[i].astype(str))
except:
    pass

#Getting the target variable aka the variable we gonna predict
#here we are using the fact that the target variable wouldn't be present in the test data
a=set(train.columns)
b=set(test.columns)
c=list(a-b)
label=c[0]
print('Target variable is :', label) #showing the target variable

#assiging dependent and independent variables
X=train.drop(label,axis=1)
y=train[label]

#train_test_split()
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.25, random_state=4)

#modelling
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier()
rf.fit(X_train,y_train)
pred_y=rf.predict(X_val)
from sklearn.metrics import accuracy_score
print('The accuracy is :',accuracy_score(y_val.values,pred_y))

#making predictions on test data
y_pred=rf.predict(test)

#saving the submissions in the form of pandas dataframe
#submission=pd.DataFrame({label:y_pred},index=test.index) : freq used in older versions
sample[label]=y_pred
file_name=input('Enter the title for saving predictions :')
submission.to_csv(file_name)
print('SUBMISSIONS SAVED SUCCESSFULLY!!!')
sample.head()#making sure that the submission is saved.

Libraries Imported


Enter the train data : ../input/spaceship-titanic/train.csv
Enter the test data : ../input/spaceship-titanic/test.csv
Enter the sample submission : ../input/spaceship-titanic/sample_submission.csv


Data Loaded!
We filled the missing values!
The Id of data is : PassengerId
Target variable is : Transported
The accuracy is : 0.7856485740570377


Enter the title for saving predictions : spaceship.csv


SUBMISSIONS SAVED SUCCESSFULLY!!!


,PassengerId,Transported
0,0013_01,False
1,0018_01,False
2,0019_01,True
3,0021_01,True
4,0023_01,True


suggested changes :
1. change the output same form as that of label in train data.

In [ ]:
#Version 5 code
#importing libraries
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')
print('Libraries Imported')

train_data=input('Enter the train data :')
#for application this has to be like upload your file kinda option
train=pd.read_csv(train_data)

test_data=input('Enter the test data :')
test=pd.read_csv(test_data)

print('Data Loaded!')

train_describe=pd.DataFrame(train.describe())
#the reason i'm saving this as the data frame is that it will only have numeric columns
#so while making changes required specifically for numeric columns it will helpful
#we will see the use of it in near future

test_describe=pd.DataFrame(test.describe())

#describing with category
#we will put it in try...except block since some of the data many not have categorical attributes
try:
    train_describe_cat=pd.DataFrame(train.describe(include=['O']))
except Exception as e:
    pass
try:
    test_describe_cat=pd.DataFrame(test.describe(include=['O']))
except Exception as e:
    pass
#We will fill these missing values -
#using median for numerical data
#using top for categorical data
train_describe_columns=list(train_describe.columns)
test_describe_columns=list(test_describe.columns)
try:
    train_describe_cat_columns=list(train_describe_cat.columns)
except:
    pass
try:
    test_describe_cat_columns=list(test_describe_cat.columns)
except:
    pass

#filling the missing values for numerical values with the median
for i in train_describe_columns:
    train[i].fillna(train[i].median(), inplace=True)
for i in test_describe_columns:
    test[i].fillna(test[i].median(), inplace=True)
    
#filling the missing values for categorical values with the mostly reccurring value
try:
    for i in train_describe_cat_columns:
        train[i].fillna(train_describe_cat.iloc[2,train_describe_cat_columns.index(i)], inplace=True)
    for i in test_describe_cat_columns:
        test[i].fillna(test_describe_cat.iloc[2,test_describe_cat_columns.index(i)], inplace=True)
except:
    pass
print('We filled the missing values!')

#setting ID's
try:
    for i in list(train.columns):
        if 'Id' in i:
            Id=i
            print('The Id of data is :', Id)
        else:
            pass
    train.set_index(Id ,inplace=True)
    test.set_index(Id ,inplace=True)
except:
    pass
#Labelling the data
try:
    from sklearn.preprocessing import LabelEncoder
    for i in train_describe_cat_columns:
        le=LabelEncoder()
        arr=np.concatenate((train[i], test[i])).astype(str)
        le.fit(arr)
        train[i]=le.transform(train[i].astype(str))
        test[i]=le.transform(test[i].astype(str))
except:
    pass

a=set(train.columns)
b=set(test.columns)
c=list(a-b)
label=c[0]
print('Target variable is :', label)

X=train.drop(label,axis=1)
y=train[label]

from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.25, random_state=4)

from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier()
rf.fit(X_train,y_train)
pred_y=rf.predict(X_val)
from sklearn.metrics import accuracy_score
print('The accuracy is :',accuracy_score(y_val.values,pred_y))

y_pred=rf.predict(test)

submission=pd.DataFrame({label:y_pred},index=test.index)
file_name=input('Enter the title for saving predictions :')
submission.to_csv(file_name)
print('SUBMISSIONS SAVED SUCCESSFULLY!!!')